# JNMongo の利用方法

## JNMongo とはなにか
JNMongo とは Japanese Niconico Parser with Mongo DB の略で、ニコニコ大百科のデータセットを、自然言語処理分野などの分野で用いることの出来る「知識」へと変換するためのツールです。

## JNMongo の目的
このツールを利用する、あるいはこのツールのレポジトリに対して機能を提案、追加してもらうことで知識化の知見を集め、より本研究・開発分野を発展させることがこのツールの目的です。


# JNMongo を初めてみる
まずシンプルな例として、Web上のニコニコ大百科のページを **一つ**、処理してみます。

## 目次
1. WebページをJSON化する
2. JSONファイルを読み込む
3. JSONをより細かく処理する
    1. タイトルの抽出
    2. キーワードの抽出
    3. 関連項目の抽出
    4. 概要の抽出
    5. 記事の分割
4. MongoDB へ保存する
    1. タイトルを保存する
    2. キーワードを保存する
    2. 関連項目を保存する
    3. 概要を保存する
    4. 記事をセクションごとに保存する

## Web ページをJSON化する
一般的なWebページの解析には、Beautiful Soup を用いることが一般的ですが、この目的では次の理由で適用が不適切であると考えられます。
- 特定のキーワードを抽出したいわけではないため、記事全体を俯瞰したい
- a タグのような重要なタグを逃してしまう

そのため、一度 HTML を JSON 形式に変換して、アクセスと解析を容易にします。    
JSON化するためのツールは、https://github.com/MokkeMeguru/niconico-parser　にあります。    
この部分に関しては DSL （ドメイン特化言語）の扱いに長けた Lisp で記述しています。

In [1]:
# !lein par[se-from-web -u https://dic.nicovideo.jp/a/<contents-title>

今回は先に用意した https://github.com/MokkeMeguru/jnmongo/blob/master/三枝明那.json を用いて解析してみます。

まずこの JSON ファイルの中身の構成を確認します。

```
--- <...>.json
 |- title            :単語そのもの
 |- title-head       :記事自体のタイトル
 |- article          :記事の中身 (ヘッダやフッタ、広告を除く)
 |- related_words    :関連項目部分
 |- keywords         :単語に関わるキーワードのリスト
```

In [1]:
import pandas as pd
from parse import json_parser
from pathlib import Path
raw_data = json_parser.read_json(Path("./三枝明那.json"))
keys = ["title", "title-head", "related_words", "keywords", "article"]

data = {}
for k in keys:
    data[k] = str(raw_data[k])
pd.DataFrame(data.values(), index=data.keys(), columns=["value"])

,value
title,三枝明那
title-head,三枝明那とは (サエグサアキナとは) [単語記事] - ニコニコ大百科
related_words,"[[{'content': ['にじさんじ', 'ベルモンド・バンデラス（尊敬するライバー）..."
keywords,"['三枝明那', 'サエグサアキナ', 'にじさんじ', 'ショタコン', 'エクス・アルビ..."
article,"[{'content': [{'content': ['概要'], 'tag': 'li',..."


## JSONデータを更にパースする
JSON化したデータを更に細かく分離して扱いやすくします。

### タイトルの抽出

In [2]:
from service import utils
title = utils.get_element(raw_data, "title")
title

'三枝明那'

### キーワードの抽出
その単語に関連するキーワードを抽出します。

In [3]:
keywords = utils.get_element(raw_data, "keywords", message="this article has no keywords")
keywords

['三枝明那', 'サエグサアキナ', 'にじさんじ', 'ショタコン', 'エクス・アルビオ', 'コラボ', 'デビュー', '愛園愛美']

### 関連項目の抽出
関連項目は、語のリストとして抽出する場合が便利ですが、テーブル構造情報などを用いることでより高品質な情報を抽出できる可能性があります。

In [4]:
from service import related_words
raw_related_words, related_words = related_words.extract_item(raw_data)

data = {
    "raw": str(raw_related_words),
    "extract": str(related_words[:5]),
    "#related_words": len(related_words)
}
pd.DataFrame(data.values(), index=data.keys(), columns=["value"])

,value
raw,"[[{'content': ['にじさんじ', 'ベルモンド・バンデラス（尊敬するライバー）..."
extract,"['日本武道館', '🎨鈴原るる', '♨小野町春香', '🏰御伽原江良', '📷瀬戸美夜子']"
#related_words,32


### 概要の抽出
記事のはじめと <概要> 部にある内容は、その単語の概要になっています。    
残りは left として次の処理に回します。

In [5]:
from service import absts
article = utils.get_element(raw_data, "article", message="this article has no body")
abstruct, _, left = absts.extract_abstruct(article)
str(abstruct)[:200]+"..."

"[['三枝明那（さえぐさ あきな）とは、いちから株式会社が運営する「にじさんじ」所属のバーチャルライバーである。'], {'content': ['概要'], 'tag': 'h2', 'attrs': {'id': 'h2-1'}, 'type': 'element'}, [{'content': [{'content': [{'content': [{'content': ['バーチャルライバ..."

### 記事の分割
残りの記事 left を項目 (section) ごとに分割しておきます。    
こうすることで、問題を SQuAD タスクなどで実現可能なレベルの文字長に落ち着かせることが出来ます。    

それぞれ項目のタイトルは階層構造になっている場合を考え、リストになっています。

e.g.
```
<h2>hoge</h2>
  <p>body1</p>
  <h3>bar</h3>
    <p>body2</p>
<h2>foo<h2>
  <h2>boo</h2>
```
⇒
```
hoge               ... [hoge]
  |- body1
  |- bar           ... [hoge, bar]
      |- body2
foo                ... [foo]
  |- boo
```

In [6]:
from service import contents
sections, _ = contents.SectionParser().parse(left)

data = {}
for i, sec in enumerate(sections):
    data[str(sec.titles)] = str(sec.contents)
pd.DataFrame(data.values(), index=data.keys(), columns=["value"])

,value
"('関連リンク',)",({'content': [{'content': ['三枝明那 – にじさんじ 公式サイト...
"('関連項目',)","({'content': [{'content': ['にじさんじ'], 'tag': 'l..."
"('脚注',)",({'content': [{'content': ['*そもそも出演の経緯は彼女も誤って「...


# MongoDB へ保存する
解析したものを MongoDB へ保存します。
MongoDB を用いている理由は、データの形式がJSONであることとの親和性が主な理由です。

**required** : MongoDB のセットアップ

### PyMongo の準備
MongoDB との接続は Python を用いたいために、 PyMongo を用います。Python ↔ MongoDB の接続を次のようにして確立します。    

In [7]:
from getpass import getpass
from pymongo import MongoClient

password = getpass()
username = 'root'
client = MongoClient(host="127.0.0.1",
                    port=27017,
                    username=username,
                    password=password)

次に、それぞれの保存先 (Collection) への接続を補助するクライアントインスタンスを作ります。   
初回時 (or reset 時) には、それぞれの保存先が空であることが保証されています。

In [8]:
from boundary.absts import Abst
from boundary.contents import Content
from boundary.dockeywords import DocKeywords
from boundary.keywords import Keyword
from boundary.related_words import Related_Words
abst_db = Abst(client)
content_db = Content(client)
keyword_db = Keyword(client)
dockeywords_db = DocKeywords(client)
relatedwords_db = Related_Words(client)
abst_db.reset()
content_db.reset()
keyword_db.reset()
dockeywords_db.reset()
relatedwords_db.reset()

In [10]:
data = {
    "abst": str(abst_db.all),
    "content" : str(content_db.all),
    "keyword": str(keyword_db.all),
    "keywords_by_document": str(dockeywords_db.all),
    "relatedwords": str(relatedwords_db.all)
}
pd.DataFrame(data.values(), index=data.keys(), columns=["value"])

,value
abst,[]
content,[]
keyword,[]
keywords_by_document,[]
relatedwords,[]


## タイトルを保存する
タイトルやキーワード、などの単語に当たるものは全て、同一の ObjectId で管理します。こうすることで検索性能が向上します。

In [9]:
keyword_db.insert(title)
print("{} -> {}".format(title, keyword_db.find_object(title)))

三枝明那 -> 5e65e0dcbec0480af8aef1cc


## キーワードを保存する

In [10]:
for keyword in keywords:
    keyword_db.insert(keyword)
keyword_db.all[:2]

[{'_id': ObjectId('5e65e0dcbec0480af8aef1cc'),
  'keyword': '三枝明那',
  'insertion_date': datetime.datetime(2020, 3, 9, 6, 23, 24, 302000),
  'last_update_date': datetime.datetime(2020, 3, 9, 6, 23, 25, 800000),
  'reference': 2},
 {'_id': ObjectId('5e65e0ddbec0480af8aef1d2'),
  'keyword': 'サエグサアキナ',
  'insertion_date': datetime.datetime(2020, 3, 9, 6, 23, 25, 803000),
  'last_update_date': datetime.datetime(2020, 3, 9, 6, 23, 25, 803000),
  'reference': 1}]

これらのキーワードは特定の単語タイトルについて紐付けられているので、このリレーションも保存します。

In [11]:
dockeywords_db.insert(
        keyword_db.find_object(title),
        [keyword_db.find_object(keyword) for keyword in keywords])

In [12]:
dockeywords_db.all

[{'_id': ObjectId('5e65e0dcbec0480af8aef1cc'),
  'insertion_date': datetime.datetime(2020, 3, 9, 6, 23, 27, 301000),
  'keywords': [ObjectId('5e65e0dcbec0480af8aef1cc'),
   ObjectId('5e65e0ddbec0480af8aef1d2'),
   ObjectId('5e65e0ddbec0480af8aef1d5'),
   ObjectId('5e65e0ddbec0480af8aef1d8'),
   ObjectId('5e65e0ddbec0480af8aef1db'),
   ObjectId('5e65e0ddbec0480af8aef1de'),
   ObjectId('5e65e0ddbec0480af8aef1e1'),
   ObjectId('5e65e0ddbec0480af8aef1e4')],
  'last_update_date': datetime.datetime(2020, 3, 9, 6, 23, 27, 301000)}]

## 関連項目を保存する

In [13]:
for word in related_words:
    keyword_db.insert(word)
    
relatedwords_db.insert(
    keyword_db.find_object(title),
    [keyword_db.find_object(word) for word in related_words],
    raw_related_words)

In [14]:
relatedwords_db.all

[{'_id': ObjectId('5e65e0dcbec0480af8aef1cc'),
  'contents': [[{'content': ['にじさんじ',
      'ベルモンド・バンデラス（尊敬するライバー）',
      '鈴鹿詩子/海夜叉神/轟京子（歴代のショタコンライバー）',
      '愛園愛美（一緒にデビューした同期生にしてコンビ『紅ズワイガニ』の相方）',
      '青道アカト（煩い同業者繋がり）',
      'Ni-na（同じく目標が日本武道館）',
      '日本武道館',
      'ショタコン/腐男子',
      '堀内健',
      '紅生姜/赤唐辛子',
      'バーチャルYouTuber',
      'いちから株式会社'],
     'tag': 'ul'},
    {'content': [{'content': [{'content': ['「にじさんじ」2019年上期加入ライバー'],
         'tag': 'tr'},
        {'content': ['🐺童田明治', '🧠久遠千歳', '🐽郡道美玲'], 'tag': 'tr'},
        {'content': ['🌖夢月ロア', '♨小野町春香', '🧂語部紡'], 'tag': 'tr'},
        {'content': ['📷瀬戸美夜子', '🏰御伽原江良', '🍹戌亥とこ'], 'tag': 'tr'},
        {'content': ['⚖アンジュ・カトリーナ', '👑リゼ・ヘルエスタ', '🌶三枝明那'], 'tag': 'tr'},
        {'content': ['💕愛園愛美', '🎨鈴原るる', '🌐雪城眞尋'], 'tag': 'tr'},
        {'content': ['🛡エクス・アルビオ', '🔲レヴィ・エリファ', '🍃葉山舞鈴'], 'tag': 'tr'},
        {'content': ['🎃ニュイ・ソシエール'], 'tag': 'tr'}],
       'tag': 'tbody'}],
     'tag': 'table'}]],
  'insertion_date': datetime.datet

## 概要を保存する

In [15]:
abst_db.insert(
    keyword_db.find_object(title),
    abstruct
)

In [16]:
str(abst_db.all)[:500] + "..."

'[{\'_id\': ObjectId(\'5e65e0dcbec0480af8aef1cc\'), \'contents\': [[\'三枝明那（さえぐさ あきな）とは、いちから株式会社が運営する「にじさんじ」所属のバーチャルライバーである。\'], {\'content\': [\'概要\'], \'tag\': \'h2\', \'attrs\': {\'id\': \'h2-1\'}, \'type\': \'element\'}, [{\'content\': [{\'content\': [{\'content\': [{\'content\': [\'バーチャルライバー\'], \'tag\': \'th\', \'attrs\': {\'colspan\': \'2\', \'style\': "border: 1px solid #fff; background: url(\'https://dic.nicovideo.jp/oekaki/823641.png\') center; color: #000; padding: 1px;"}, \'type\': \'element\'}], \'tag\': \'tr\', \'attrs\': None, \'type\': \'eleme...'

## 記事をセクションごと保存する

In [19]:
for sec in sections:
    for word in sec.titles:
        keyword_db.insert(word)

for sec in sections:
    content_db.insert(
        keyword_db.find_object(title),
        [keyword_db.find_object(title) for title in sec.titles],
        sec.contents)

In [20]:
content_db.all[:2]

[{'_id': ObjectId('5e65e10f2021f75877f06d93'),
  'doc_title': ObjectId('5e65e0dcbec0480af8aef1cc'),
  'child_titles': [ObjectId('5e65e0e8bec0480af8aef27a')],
  'contents': [{'content': [{'content': ['三枝明那 – にじさんじ 公式サイト'],
      'tag': 'li',
      'attrs': None,
      'type': 'element'},
     {'content': ['三枝明那(さえぐさあきな) - にじさんじ Wiki*'],
      'tag': 'li',
      'attrs': None,
      'type': 'element'}],
    'tag': 'ul',
    'attrs': None,
    'type': 'element'}]},
 {'_id': ObjectId('5e65e10f2021f75877f06d94'),
  'doc_title': ObjectId('5e65e0dcbec0480af8aef1cc'),
  'child_titles': [ObjectId('5e65e0e8bec0480af8aef27d')],
  'contents': [{'content': [{'content': ['にじさんじ'],
      'tag': 'li',
      'attrs': None,
      'type': 'element'},
     {'content': ['ベルモンド・バンデラス（尊敬するライバー）'],
      'tag': 'li',
      'attrs': None,
      'type': 'element'},
     {'content': ['鈴鹿詩子/海夜叉神/轟京子（歴代のショタコンライバー）'],
      'tag': 'li',
      'attrs': None,
      'type': 'element'},
     {'content': ['愛園愛美（一緒にデビューし

以上が現状サポートされているパース・保存の流れとなります。
次は JNMongo_multifile.ipynb で、複数ファイルでの処理について紹介します。